In [1]:
import tensorflow as tf

In [2]:
class Inception_Block(tf.keras.layers.Layer):
    def __init__(self, 
                 conv_1, 
                 conv_3_red, 
                 conv_3, 
                 conv_5_red,
                 conv_5,
                 conv_pool_proj
                ):
        super(Inception_Block, self).__init__()
        self.conv_1 = conv_1
        self.conv_3_red = conv_3_red
        self.conv_3 = conv_3
        self.conv_5_red = conv_5_red
        self.conv_5 = conv_5
        self.conv_pool_proj = conv_pool_proj
        
        self.Conv_1 = tf.keras.layers.Conv2D(filters = self.conv_1,
                                             kernel_size = 1,
                                             padding = 'same',
                                             activation = 'relu'
                                            )
        self.Conv_3_red = tf.keras.layers.Conv2D(filters = self.conv_3_red,
                                               kernel_size = 1,
                                               padding = 'same',
                                               activation = 'relu'
                                              )
        self.Conv_3 = tf.keras.layers.Conv2D(filters = self.conv_3,
                                               kernel_size = 3,
                                               padding = 'same',
                                               activation = 'relu'
                                              )
        self.Conv_5_red = tf.keras.layers.Conv2D(filters = self.conv_5_red,
                                               kernel_size = 1,
                                               padding = 'same',
                                               activation = 'relu'
                                              )
        self.Conv_5 = tf.keras.layers.Conv2D(filters = self.conv_5,
                                               kernel_size = 5,
                                               padding = 'same',
                                               activation = 'relu'
                                              )
        self.Pool = tf.keras.layers.MaxPool2D(pool_size = 3,
                                                strides = 1,
                                                padding = 'same'
                                               )
        self.Pool_proj = tf.keras.layers.Conv2D(filters = self.conv_pool_proj,
                                               kernel_size = 1,
                                               padding = 'same',
                                               activation = 'relu'
                                              )
    
    def call(self, X):
        #1st path
        y1 = self.Conv_1(X)
        #2nd path
        y2 = self.Conv_3_red(X)
        y2 = self.Conv_3(y2)
        #3rd path
        y3 = self.Conv_5_red(X)
        y3 = self.Conv_5(y3)
        #4th path
        y4 = self.Pool(X)
        y4 = self.Pool_proj(y4)
        
        y = tf.concat([y1,y2,y3,y4], axis = -1)
        return y
    
    
class Aux_classifier(tf.keras.layers.Layer):
    def __init__(self, n_labels:int, last_activation:str):
        super(Aux_classifier, self).__init__()
        self.n_labels = n_labels
        self.last_activation = last_activation
        
        self.AP = tf.keras.layers.AveragePooling2D(pool_size = (5,5),
                                                   strides = (3,3),
                                                   padding = 'valid'
                                                  )
        self.Conv = tf.keras.layers.Conv2D(filters = 128,
                                           kernel_size = (1,1),
                                           padding = 'same',
                                           activation = 'relu'
                                          )
        self.FC = tf.keras.layers.Dense(1024,
                                         activation = 'relu'
                                        )
        self.DO = tf.keras.layers.Dropout(.7)
        self.O = tf.keras.layers.Dense(self.n_labels,
                                       activation = self.last_activation
                                      )

In [3]:
class GoogleNet(tf.keras.models.Model):
    def __init__(self, n_labels:int, last_activation:int='softmax'):
        super(GoogleNet, self).__init__()
        self.n_labels = n_labels
        self.last_activation = last_activation
        
        #Stem
        self.Conv1 = tf.keras.layers.Conv2D(filters = 64,
                                            kernel_size = 7,
                                            strides = 2,
                                            padding = 'same',
                                            activation = 'relu'
                                           )
        self.MP1 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                             strides = 2,
                                             padding = 'same'
                                            )
        self.Conv2 = tf.keras.layers.Conv2D(filters = 64,
                                            kernel_size = 1,
                                            padding = 'valid',
                                            activation = 'relu'
                                           )
        self.Conv3 = tf.keras.layers.Conv2D(filters = 192,
                                            kernel_size = 3,
                                            padding = 'same',
                                            activation = 'relu'
                                           )
        self.MP2 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                             strides = 2,
                                             padding = 'same'
                                            )
        #Inceoption and Auxiliary classifier
        self.Inception3a = Inception_Block(64, 96, 128, 16, 32, 32)
        self.Inception3b = Inception_Block(128, 128, 192, 32, 96, 64)
        self.MP3 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                             strides = 2,
                                             padding = 'same'
                                            )
        self.Inception4a = Inception_Block(192, 96, 208, 16, 48, 64)
        self.Auxiliary_Classifier1 = Aux_classifier(self.n_labels, self.last_activation)
        self.Inception4b = Inception_Block(160, 112, 224, 24, 64, 64)
        self.Inception4c = Inception_Block(128, 128, 256, 24, 64, 64)
        self.Inception4d = Inception_Block(112, 144, 288, 32, 64, 64)
        self.Auxiliary_Classifier2 = Aux_classifier(self.n_labels, self.last_activation)
        self.Inception4e = Inception_Block(256, 160, 320, 32, 128, 128)
        self.MP4 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                             strides = 2,
                                             padding = 'same'
                                            )
        self.Inception5a = Inception_Block(256, 160, 320, 32, 128, 128)
        self.Inception5b = Inception_Block(384, 192, 384, 48, 128, 128)
        #Classifier
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (7,7),
                                             strides = 1,
                                             padding = 'valid'
                                            ),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(.4),
            tf.keras.layers.Dense(self.n_labels, activation = self.last_activation)
        ])
        
    def call(self, X):
        y = self.Conv1(X)
        print(y.shape)
        y = self.MP1(y)
        print(y.shape)
        y = tf.nn.local_response_normalization(y)
        y = self.Conv2(y)
        y = self.Conv3(y)
        print(y.shape)
        y = tf.nn.local_response_normalization(y)
        y = self.MP2(y)
        print(y.shape)
        y = self.Inception3a(y)
        print(y.shape)
        y = self.Inception3b(y)
        print(y.shape)
        y = self.MP3(y)
        print(y.shape)
        y = self.Inception4a(y)
        print(y.shape)
        y_aux1 = self.Auxiliary_Classifier1(y)
        y = self.Inception4b(y)
        print(y.shape)
        y = self.Inception4c(y)
        print(y.shape)
        y = self.Inception4d(y)
        print(y.shape)
        y_aux2 = self.Auxiliary_Classifier2(y)
        y = self.Inception4e(y)
        print(y.shape)
        y = self.MP4(y)
        print(y.shape)
        y = self.Inception5a(y)
        print(y.shape)
        y = self.Inception5b(y)
        print(y.shape)
        y = self.classifier(y)
        return y, y_aux1, y_aux2

In [4]:
gn = GoogleNet(1000, 'softmax')

In [5]:
gn.build([None,224,224,3])

(None, 112, 112, 64)
(None, 56, 56, 64)
(None, 56, 56, 192)
(None, 28, 28, 192)
(None, 28, 28, 256)
(None, 28, 28, 480)
(None, 14, 14, 480)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 528)
(None, 14, 14, 832)
(None, 7, 7, 832)
(None, 7, 7, 832)
(None, 7, 7, 1024)


In [6]:
gn.summary()

Model: "google_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4160      
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  110784    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
inception__block (Inception_ multiple                  163696    
_________________________________________________________________
inception__block_1 (Inceptio multiple                  3